In [2]:
import numpy as np

# Q1 - State Representation

In [3]:
# Motion model (unicycle) - f
def  f(control_input, dt, initial_pose):
  velocity, angular_momentum = control_input
  x_initial, y_initial, theta_initial = initial_pose
  return np.hstack([
    x_initial + velocity * dt * np.cos(theta_initial),
    y_initial + velocity * dt * np.sin(theta_initial),
    theta_initial + angular_momentum * dt
    ])
# Origin pose
x_0 = np.hstack([0.0, 0.0, 0.0])
# Next pose (predicted)
u_t = np.hstack([1.0, 0.3])
dt = 0.1
x_pred = f(u_t, dt, x_0)
print("Pose prediction: ", x_pred)
# Distance moved
position_change = (x_pred - x_0)[:1]
distance_moved = np.linalg.norm(position_change)
print("Distance moved: ", distance_moved)


Pose prediction:  [0.1  0.   0.03]
Distance moved:  0.1


# Q2 - Prediction

In [4]:
# Analytical Jacobian, F = delta f / delta x
def F_analytical(u, dt, x):
  velocity, _ = u
  _, _, theta = x
  return np.array([
  [1.0, 0.0, -velocity * dt * np.sin(theta)],
  [0.0, 1.0, velocity * dt * np.cos(theta)],
  [0.0, 0.0, 1.0]
  ])
# Return the Jacobian at the current state
F_a = F_analytical(u, dt, x_0)
print("Analytical F_t: \n", F_a)
# Calculate the numerical Jacobian
eps = 1e-6
def F_numeric(u, dt, x, eps):
  F_n = np.zeros((3, 3))
  for i in range(3):
    dx = np.zeros(3)
    dx[i] = eps
    F_n[:, i] = (f(u, dt, x + dx) - f(u, dt, x - dx)) / (2 * eps)
  return F_n
F_n = F_numeric(u, dt, x_0, eps)
print("Numerical F_t: \n", F_n)
# Compare Jacobians
print("Difference F (numeric - analytical):\n", np.round(F_n - F_a, 4))
# Process noise covariance
Q_t = np.diag([0.01, 0.01, 0.001])
# Initial covariance
P_0 = np.diag([0.1, 0.1, 0.05])
# Predicted covariance
P_pred = F_a @ P_0 @ F_a.T + Q_t


NameError: name 'u' is not defined

# Q3 - Measurement Update

In [ ]:
# Measurement model (range-bearing method) - h
def h(x_pred, m):
  x, y, theta = x_pred
  m_x, m_y = m
  r = np.sqrt((m_x - x)**2 + (m_y - y)**2)
  phi = np.arctan((m_y - y)/(m_x - x)) - theta
  return np.array([r, phi])
# Landmark 1
m1 = np.array([2.0, 1.0])
# Predicted measurement
z_pred = h(x_pred, m1)
print("Predicted measurement: ", np.round(z_pred, 3))
# H_t measurement Jacobian evaluated at X_t^ - analytical
def H_analytical(x_pred, m):
  x, y, _ = x_pred
  m_x, m_y = m
  delta_x = m_x - x
  delta_y = m_y - y
  q = delta_x**2 + delta_y**2
  r = np.sqrt(q)
  H = np.zeros((2, 3))
  H[0, 0] = -delta_x / r
  H[0, 1] = -delta_y / r
  H[1, 0] = delta_y / q
  H[1, 1] = -delta_x / q
  H[1, 2] = -1
  return H
# Return H_t_a
H_t_a = H_analytical(x_pred, m1)
print("Measurement Jacobian (analytical): \n", np.round(H_t_a, 3))
# H_t measurement Jacobian evaluated at X_t^ - numerical
eps = 1e-6
def H_numerical(x_pred, m, eps):
  H = np.zeros((2, 3))
  for i in range(3):
    dx = np.zeros(3)
    dx[i] = eps
    H[:, i] = (h(x_pred + dx, m) - h(x_pred - dx, m)) / (2*eps)
  return H
# Return H_t_n
H_t_n = H_numerical(x_pred, m1, eps)
print("Measurement Jacobian (numerical): \n", np.round(H_t_n, 3))
# Compare Measurement Jacobian
print("Measurement Jacobian (compare): \n", np.round(H_t_a - H_t_n, 3))
# Measurement noise covariance
R_t = np.diag([0.1**2, (np.pi/180)**2])
# Innovation covariance
S_t = H_t_a @ P_pred @ H_t_a.T + R_t
print("Innovation covariance: \n", S_t)

Predicted measurement:  [2.147 0.454]
Measurement Jacobian (analytical): 
 [[-0.885 -0.466  0.   ]
 [ 0.217 -0.412 -1.   ]]
Measurement Jacobian (numerical): 
 [[-0.885 -0.466  0.   ]
 [ 0.217 -0.412 -1.   ]]
Measurement Jacobian (compare): 
 [[ 0. -0.  0.]
 [ 0. -0.  0.]]
Innovation covariance: 
 [[0.12010846 0.00242471]
 [0.00242471 0.0793722 ]]


# Q4 - Update Step

In [ ]:
# Actual measurement
z_obs = np.array([2.2, 0.45])
print("Measurement observation: ", z_obs)
# Innovation residual
r_t = z_obs - z_pred
print("Innovation (residual): ", r_t)
# Kalman gain
K_t = P_pred @ H_t_a.T @ np.linalg.inv(S_t)
print("Kalman gain: \n", K_t)
# Posterior mean (update estimate)
x_update = x_pred + K_t @ r_t
print("Posterior mean: \n", x_update)
# Posterior coveriance (update uncertainty)
P_update = (np.eye(3) - K_t @ H_t_a) @ P_pred
print("Posterior covariance: \n", P_update)

Measurement observation:  [2.2  0.45]
Innovation (residual):  [ 0.05290894 -0.00447793]
Kalman gain: 
 [[-0.81701529  0.32558248]
 [-0.41588902 -0.62407104]
 [-0.00589665 -0.66832521]]
Posterior mean: 
 [ 0.05531465 -0.0192097   0.03268073]
Posterior covariance: 
 [[ 0.02270213 -0.02559199  0.01537304]
 [-0.02559199  0.05755429 -0.02908216]
 [ 0.01537304 -0.02908216  0.01552444]]


# Q5 - Extension to 3D via Camera Projection Theorem

In [10]:
# Camera intrinsic model
def camera_intrinsitcs(fx, fy, cx, cy):
  return np.array([
  [fx, 0, cx],
  [0, fy, cy],
  [0.0, 0.0, 1.0]
])
K = camera_intrinsitcs(800, 800, 320, 240)
# Euler rotation to rotation matrix
def euler_to_R(roll, pitch, yaw):
    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(roll), -np.sin(roll)],
        [0, np.sin(roll), np.cos(roll)]
    ])
    Ry = np.array([
        [np.cos(pitch), 0, np.sin(pitch)],
        [0, 1, 0],
        [-np.sin(pitch), 0, np.cos(pitch)]
    ])
    Rz = np.array([
        [np.cos(yaw), -np.sin(yaw), 0],
        [np.sin(yaw), np.cos(yaw), 0],
        [0, 0, 1]
    ])
    # ZYX order (yaw-pitch-roll)
    return Rz @ Ry @ Rx
# Measurement model - camera
def h_camera(x_cam, p_W, K):
  # Unpack
  x, y, z, roll, pitch, yaw = x_cam
  fx = K[0, 0]
  cx = K[0, 2]
  fy = K[1, 1]
  cy = K[1, 2]
  # Convert
  t_CW = np.array([x, y, z])
  R_CW = euler_to_R(roll, pitch, yaw)
  # Camera projection theorem
  p_C = R_CW @ (p_W - t_CW)
  Xc, Yc, Zc = p_C
  # Camera coordinates to image pixel coordinates
  u = fx * Xc / Zc + cx
  v = fy * Yc / Zc + cy
  return np.array([u, v])
# Measurement Jacobian - analytical
def H_camera_numeric(x_cam, p_W, K, eps=1e-6):
    H = np.zeros((2, 6))
    for i in range(6):
        dx = np.zeros(6)
        dx[i] = eps
        H[:, i] = (h_camera(x_cam + dx,  p_W, K) - h_camera(x_cam - dx,  p_W, K)) / (2*eps)
    return H
# Example camera pose and world point
x_cam = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
p_W = np.array([1.0, 0.5, 5.0])
# Predicted pixel measurement
z_pred = h_camera(x_cam, p_W, K)
print("Predicted pixel:", np.round(z_pred, 2))
# Measurement Jacobian
H_num = H_camera_numeric(x_cam, p_W, K)
print("Numeric Jacobian H:\n", np.round(H_num, 5))

Predicted pixel: [480. 320.]
Numeric Jacobian H:
 [[-160.    0.   32.  -16.  832.  -80.]
 [   0. -160.   16. -808.   16.  160.]]
